In [1]:
import pandas as pd
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import random
import pandas as pd
import seaborn as sns
from collections import Counter, defaultdict
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
df= pd.read_pickle("data/features.pkl")
df.head()

,original_text,label,preprocessed,word_count,avg_word_count,syllable_count,uncommon,difficult_words,stem,discourse,cohesive_features,flesch,dale,mcalpine,nouns_adjs,normalized
0,There is manuscript evidence that Austen conti...,1,"[there, is, manuscript, evidence, that, austen...",35,4.485714,1.371429,14,7,there is manuscript evid that austen continu t...,4,2,52.87,11.24,48.0,0.228571,there is manuscript evidence that austen conti...
1,"In a remarkable comparative analysis , Mandaea...",1,"[in, a, remarkable, comparative, analysis, man...",19,6.000000,1.789474,14,8,there is manuscript evid that austen continu t...,2,1,35.27,14.55,23.0,0.315789,in a remarkable comparative analysis mandaean ...
2,"Before Persephone was released to Hermes , who...",1,"[before, persephone, was, released, to, hermes...",40,4.725000,1.400000,15,9,there is manuscript evid that austen continu t...,7,3,47.80,11.15,57.0,0.175000,before persephone was released to hermes who h...
3,Cogeneration plants are commonly found in dist...,1,"[cogeneration, plants, are, commonly, found, i...",32,6.281250,1.781250,22,14,there is manuscript evid that austen continu t...,0,1,22.08,14.60,38.0,0.593750,cogeneration plants are commonly found in dist...
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1,"[geneva, is, the, city, in, switzerland, after...",20,4.650000,1.350000,7,4,there is manuscript evid that austen continu t...,0,2,68.10,8.58,29.0,0.400000,geneva is the city in switzerland after zürich...


### Split dataset into test, train and dev

We have a large enough dataset that we can effectively split it into train, development, and test sets, using the standard ratio of 80%, 10%, 10% for each, respectively. We'll use `split` from `numpy` to split the data into train, dev, and test separately. We'll call these `train_df`, `dev_df`, and `test_df`. It's important to note that `split` does not shuffle, so we'll use `DataFrame.sample()` and randomly resample our entire dataset to get a random shuffle before the split.

In [3]:
RANDOM_SEED= 42

In [4]:
# create a text column that holds our clean data set in a string format
df['text'] = [' '.join(map(str, l)) for l in df['preprocessed']]

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit # helps ensure that classes are balanced before testing
X = df[['text']].values
y = df[['label']].values

s = StratifiedShuffleSplit(n_splits=1, test_size =0.2, random_state=42)  

for train_index, test_index in s.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [6]:
len(y_train),len(y_test)

(322886, 80722)

### Convert text data to features

We'll be using a `TfIdfVectorizer` to convert the text into features.

Tfidf is a numerical statistic that is meant to reflect how important a word is to a document in a corpus. The tf–idf score increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word. This helps to adjust for the words that appear more frequently in general. 

I hypothesize that using tf-idf rarer words which are often harder to comprehend will have higher weights than easier words that appear more frequently.  

In [7]:
# intitialize our vectorizer
vectorizer= TfidfVectorizer(max_features=10000,ngram_range=(1,2)) # set max vocab to 10000

X_train_tfidf = vectorizer.fit_transform(X_train.flatten())
X_test_tfidf = vectorizer.transform(X_test.flatten())


In [8]:
# check the length of the vectors
len(vectorizer.vocabulary_)

10000

In [91]:
# check the shape of our data
X_train_tfidf.shape, X_test_tfidf.shape,

((322886, 10000), (80722, 10000))

### Fit the classifier on a subset of the data

Finally, let's fit the classifier. For a start we'll use `LogisticRegression`.To train our classifier, we first create a `LogisticRegression` object and call `fit` passing in `X_train` and `y_train`.

For this cell, let's just use the first 10,000 rows of `X_train` and `y_train` to fit the classifier. 

In general, when we have a large dataset, it's useful to go end-to-end and train one of these half-baked classifiers to verify that your model works as expected. 

*Notes:*
1. We use the `lbfgs` solver, as this generally Just Works™ and is fast.
2. `X_train` is a numpy array, so you'll need to use array indexing operations to get the first 10,000 rows.

In [92]:
clf=LogisticRegression(solver="lbfgs",random_state=RANDOM_SEED)

clf.fit(X_train_tfidf[:10000],np.array(y_train)[:10000])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Create Dummy classifiers

It's always important to contextualize our results by comparing it with naive classifiers. If these classifiers do well, then your task is easy! If not, then you can see how much better your system does at first. We'll use two different strategies using the [Dummy Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html) class. Create two `DummyClassifier` instances that use the `uniform` (guess randomly) and `most_frequent` strategies and fit these on the training data so we can compare them with our regressor that was trained on 10K instances. In general, you probably always want to at least compare with these two baselines in a classification task.

In [94]:
# YOUR CODE HERE

duni=DummyClassifier(strategy="uniform",random_state=RANDOM_SEED)
dmos=DummyClassifier(strategy="most_frequent",random_state=RANDOM_SEED)

duni.fit(X_train_tfidf[:10000],np.array(y_train)[:10000])
dmos.fit(X_train_tfidf[:10000],np.array(y_train)[:10000])

DummyClassifier(constant=None, random_state=42, strategy='most_frequent')

### Let's generate our predictions

In [95]:
clf_dev_preds=clf.predict(X_test_tfidf)
duni_dev_preds=duni.predict(X_test_tfidf)
dmos_dev_preds=dmos.predict(X_test_tfidf)

### Score our predictions

In [96]:
lr_f1 = f1_score(y_test, clf_dev_preds)
rand_f1 = f1_score(y_test, duni_dev_preds)
mf_f1 = f1_score(y_test, dmos_dev_preds)

In [97]:
print(lr_f1)
print(rand_f1)
print(mf_f1)

0.6666122943561542
0.5066888608741417
0.6765800475448808


## Train on the entire data set

In [98]:
clf2=LogisticRegression(solver="lbfgs",random_state=RANDOM_SEED)

clf2.fit(X_train_tfidf,np.array(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
clf2_test_preds=clf2.predict(X_test_tfidf)

In [101]:
clf2_f1 = f1_score(y_test, clf2_test_preds)
print(clf2_f1)

0.7071695940109416


Let's try and remove stop words and see if that helps things

# Logistic regression with stop words removed

In [103]:
X_train, X_test, y_train,y_test = train_test_split(df['text'], df['label'], 
                                                    test_size=0.2, random_state=42)

In [104]:
# intitialize our vectorizer
vect= TfidfVectorizer(max_features=10000,stop_words='english',ngram_range=(1,2)) # set max vocab to 10000

In [105]:
vect.fit(df['text'])

Train_X_Tfidf = vect.transform(X_train)
Test_X_Tfidf = vect.transform(X_test)

In [106]:
clf3=LogisticRegression(solver="lbfgs",random_state=RANDOM_SEED)

clf3.fit(Train_X_Tfidf,np.array(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [107]:
clf3_test_preds=clf3.predict(Test_X_Tfidf)

In [108]:
clf3_f1 = f1_score(y_test, clf3_test_preds)
print(clf2_f1)

0.7071695940109416


# Random Forest

In [111]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(bootstrap= True,
 max_depth=50,
 max_features='auto',
 min_samples_leaf=4,
 min_samples_split=10,
 n_estimators=800)

rf.fit(Train_X_Tfidf,y_train)
rf_preds = rf.predict(Test_X_Tfidf)
rf_acc = accuracy_score(y_test, rf_preds)
rf_f1 = f1_score(y_test, rf_preds)
rf_acc, rf_f1

(0.6023636678972275, 0.6970000188796799)

# SVM

In [112]:
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
SVM = svm.SVC(C=0.5, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)

pred_SVM = SVM.predict(Test_X_Tfidf)
svm_acc = accuracy_score(y_test, pred_SVM)
svm_f1 = f1_score(y_test, predictions_SVM)
svm_acc, svm_f1